## Settings

In [1]:
# 一つ上の階層からモジュールを参照できるようにする
import sys
sys.path.append('..')

In [2]:
# モジュールの変更を自動的に反映する
%load_ext autoreload
%autoreload 2

In [38]:
import numpy as np
import pandas as pd
from inputs import load_data
# # XGB
from models import XGB
# Torch
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from models import TrainingDataset, Torch, transform_labels, restore_labels, train_model, validate_model, \
                   TestDataset, predict_test
# Keras
from keras import Sequential, Input
from keras.utils import to_categorical
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

## XGB

In [106]:
train_x, train_y, test_x = load_data()

In [107]:
params = None
model = XGB(params)

In [108]:
tr_x = train_x#[:80]
va_x = train_x#[80:]
tr_y = train_y#[:80]
va_y = train_y#[80:]
# tr_x = np.array([0 for _ in range(len(train_x))])
# tr_x = tr_x.reshape(-1, 1)
# va_x = tr_x
# tr_y = np.array([0 for _ in range(len(train_y))])
# va_y = tr_y

In [100]:
from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(tr_x, tr_y, va_x, va_y)

[15:01:03] WARNING: /Users/travis/build/dmlc/xgboost/src/predictor/cpu_predictor.cc:206: Ignoring the base margin, since it has incorrect length. The base margin must be an array of length [num_class] * [number of data points], i.e. 6 * 107 = 642. Instead, all data points will use base_score = 0.5
[15:01:03] WARNING: /Users/travis/build/dmlc/xgboost/src/predictor/cpu_predictor.cc:206: Ignoring the base margin, since it has incorrect length. The base margin must be an array of length [num_class] * [number of data points], i.e. 6 * 107 = 642. Instead, all data points will use base_score = 0.5


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [111]:
early_stopping = 5
model.fit(tr_x, tr_y, va_x, va_y,
          early_stopping_rounds=early_stopping,
          verbose=True)

[0]	validation_0-mlogloss:1.27429	validation_1-mlogloss:1.27429
Multiple eval metrics have been passed: 'validation_1-mlogloss' will be used for early stopping.

Will train until validation_1-mlogloss hasn't improved in 5 rounds.
[1]	validation_0-mlogloss:0.98543	validation_1-mlogloss:0.98543
[2]	validation_0-mlogloss:0.77405	validation_1-mlogloss:0.77405
[3]	validation_0-mlogloss:0.61166	validation_1-mlogloss:0.61166
[4]	validation_0-mlogloss:0.49693	validation_1-mlogloss:0.49693
[5]	validation_0-mlogloss:0.40317	validation_1-mlogloss:0.40317
[6]	validation_0-mlogloss:0.33250	validation_1-mlogloss:0.33250
[7]	validation_0-mlogloss:0.27737	validation_1-mlogloss:0.27737
[8]	validation_0-mlogloss:0.23674	validation_1-mlogloss:0.23674
[9]	validation_0-mlogloss:0.20367	validation_1-mlogloss:0.20367
[10]	validation_0-mlogloss:0.17655	validation_1-mlogloss:0.17655
[11]	validation_0-mlogloss:0.15712	validation_1-mlogloss:0.15712
[12]	validation_0-mlogloss:0.13919	validation_1-mlogloss:0.13919

/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


[23]	validation_0-mlogloss:0.06549	validation_1-mlogloss:0.06549
[24]	validation_0-mlogloss:0.06355	validation_1-mlogloss:0.06355
[25]	validation_0-mlogloss:0.06168	validation_1-mlogloss:0.06168
[26]	validation_0-mlogloss:0.05987	validation_1-mlogloss:0.05987
[27]	validation_0-mlogloss:0.05840	validation_1-mlogloss:0.05840
[28]	validation_0-mlogloss:0.05697	validation_1-mlogloss:0.05697
[29]	validation_0-mlogloss:0.05579	validation_1-mlogloss:0.05579
[30]	validation_0-mlogloss:0.05449	validation_1-mlogloss:0.05449
[31]	validation_0-mlogloss:0.05361	validation_1-mlogloss:0.05361
[32]	validation_0-mlogloss:0.05269	validation_1-mlogloss:0.05269
[33]	validation_0-mlogloss:0.05189	validation_1-mlogloss:0.05189
[34]	validation_0-mlogloss:0.05107	validation_1-mlogloss:0.05107
[35]	validation_0-mlogloss:0.05030	validation_1-mlogloss:0.05030
[36]	validation_0-mlogloss:0.04951	validation_1-mlogloss:0.04951
[37]	validation_0-mlogloss:0.04877	validation_1-mlogloss:0.04877
[38]	validation_0-mloglos

In [110]:
preds = model.predict(test_x)

In [ ]:
index = test_x.index.to_numpy().reshape(-1, 1)
preds = preds.reshape(-1, 1)
submission = np.concatenate((index, preds), axis=1)
submission = pd.DataFrame(submission)

In [13]:
submission.to_csv('xgb.csv', index=False, header=False)

## Torch

In [39]:
dataset = TrainingDataset(target_transform = transform_labels)

In [40]:
train_ratio = 0.95
train_size = int(train_ratio * len(dataset))
test_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, test_size])

In [41]:
batch_size = 10

train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

for X, y in val_dataloader:
    print('Shape of X', X.shape)
    print('Shape of y', y.shape, y.dtype)
    break

Shape of X torch.Size([6, 9])
Shape of y torch.Size([6]) torch.int64


In [42]:
input_size = 9
output_size = 6
model = Torch(input_size, output_size)

In [43]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [44]:
epochs = 5
for t in range(epochs):
    print(f'Epoch {t+1}\n-------------------------------')
    train_model(train_dataloader, model, loss_fn, optimizer)
    validate_model(val_dataloader, model, loss_fn)
print('Done!')

Epoch 1
-------------------------------
loss: 1.796193  [    0/  101]
loss: 1.775526  [   20/  101]
loss: 1.795517  [   40/  101]
loss: 1.787526  [   60/  101]
loss: 1.784987  [   80/  101]
loss: 1.875845  [  100/  101]
Test Error: 
Accuracy: 16.7%, Avg loss: 0.306299 

Epoch 2
-------------------------------
loss: 1.795148  [    0/  101]
loss: 1.775630  [   20/  101]
loss: 1.796246  [   40/  101]
loss: 1.788041  [   60/  101]
loss: 1.785111  [   80/  101]
loss: 1.871614  [  100/  101]
Test Error: 
Accuracy: 16.7%, Avg loss: 0.305924 

Epoch 3
-------------------------------
loss: 1.794282  [    0/  101]
loss: 1.775714  [   20/  101]
loss: 1.796897  [   40/  101]
loss: 1.788494  [   60/  101]
loss: 1.785205  [   80/  101]
loss: 1.868060  [  100/  101]
Test Error: 
Accuracy: 16.7%, Avg loss: 0.305613 

Epoch 4
-------------------------------
loss: 1.793576  [    0/  101]
loss: 1.775770  [   20/  101]
loss: 1.797454  [   40/  101]
loss: 1.788868  [   60/  101]
loss: 1.785265  [   80/  10

In [45]:
test_dataset = TestDataset()
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)
preds = predict_test(test_dataloader, model)

In [46]:
preds = [restore_labels(p) for p in preds]
preds = np.array(preds)

In [47]:
# preds

In [48]:
_, _, test_x = load_data()
index = test_x.index.to_numpy().reshape(-1, 1)
preds = preds.reshape(-1, 1)
submission = np.concatenate((index, preds), axis=1)
submission = pd.DataFrame(submission)

In [49]:
submission.to_csv('torch.csv', index=False, header=False)

## Keras

In [212]:
TRAINING_FOR_SUBMISSION = True
validation_split = 0.0 if TRAINING_FOR_SUBMISSION else 0.4

In [213]:
train_x, train_y, test_x = load_data()

In [214]:
num_classes = len(train_y.unique())
train_y = pd.get_dummies(train_y)

In [215]:
input_shape = (train_x.shape[1],)
model = Sequential([
    Input(shape=input_shape),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])
model.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_66 (Dense)             (None, 64)                640       
_________________________________________________________________
dense_67 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_68 (Dense)             (None, 6)                 390       
Total params: 5,190
Trainable params: 5,190
Non-trainable params: 0
_________________________________________________________________


In [216]:
learning_rate = 1e-3
optimizer = Adam(learning_rate=learning_rate)

In [217]:
batch_size = 10
epochs = 5

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.fit(train_x, train_y, batch_size=batch_size, epochs=epochs, validation_split=validation_split)

Epoch 1/5
11/11 [==============================] - 0s 852us/step - loss: 8.0530 - accuracy: 0.1682
Epoch 2/5
11/11 [==============================] - 0s 789us/step - loss: 3.1567 - accuracy: 0.3551
Epoch 3/5
11/11 [==============================] - 0s 829us/step - loss: 2.0668 - accuracy: 0.3178
Epoch 4/5
11/11 [==============================] - 0s 903us/step - loss: 1.8164 - accuracy: 0.3738
Epoch 5/5
11/11 [==============================] - 0s 933us/step - loss: 1.5992 - accuracy: 0.4112


In [218]:
classes = train_y.columns.tolist()
preds = model.predict(test_x).argmax(axis=1)
preds = [classes[pred] for pred in preds]
preds = np.array(preds)
preds = preds.reshape(-1, 1)

In [222]:
# preds

In [220]:
index = test_x.index.to_numpy().reshape(-1, 1)
submission = np.concatenate((index, preds), axis=1)
submission = pd.DataFrame(submission)

In [221]:
submission.to_csv('keras.csv', index=False, header=False)